In [1]:
# %run packages.ipynb

In [1]:
from pdf_files import get_fullpath_of_pdf_files

In [2]:
from pdf_documents import get_pdf_documents

In [3]:
from document_util import get_split_documents

In [4]:
from embedding_function import get_embedding_function

In [5]:
from chroma_db import create_db, get_similar_docs

In [6]:
from chain import get_chain_output

In [7]:
from match import run_text_match

In [8]:
from pdf_retrieval import save_pdfs_to_db, get_metadata_from_db, get_column_from_db, get_filename_list,get_polish_intro

In [9]:
# path = "./data/docs/literature1" # 1 good 
# path = "./data/docs/literature6" # 6 good 
# path = "./data/docs/literature9" # 9 good
path = "./data/docs/literature"  # all

chunk_size=1500
chunk_overlap=100
persist_directory = 'data/db/vector/huggingface_index_all'
collection_name = 'paper_index'
collection_name_abstract = 'abstract_index'
excel_file = 'data/db/repo.xlsx'
intro35_excel_file = 'data/db/repo_intro_35.xlsx'
intro4_excel_file = 'data/db/repo_intro_4.xlsx'

In [13]:
embedding_function = get_embedding_function()

### The following 4 lines only need to be run once (after the pdf is loaded for the first time)

In [24]:
pdf_files = get_fullpath_of_pdf_files(path)

In [25]:
pdf_docs = get_pdf_documents(pdf_files)

In [37]:
split_docs = get_split_documents(pdf_docs, chunk_size, chunk_overlap)

In [ ]:
create_db(split_docs,embedding_function,collection_name,persist_directory)

### 生成标题+润色 vector DB

In [43]:
save_pdfs_to_db(pdf_files, excel_file, is_Intro=False, pages=1)

get meta from LLM ./data/docs/abstracts/Political extremism and economic activity.pdf
Data append to  data/db/repo.xlsx
get meta from LLM ./data/docs/abstracts/Economic growth and political extremism.pdf
Data append to  data/db/repo.xlsx


In [14]:
abstract_docs = get_column_from_db(excel_file, 'abstract')

In [50]:
create_db(abstract_docs,embedding_function,collection_name_abstract,persist_directory)


Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index_all


In [14]:
# dict = get_metadata_from_db(excel_file)

## Query

### 标题+润色 embedding abstract, keywords, title 匹配相似文章，结构或是语言润色

In [15]:
my_abstract = "This paper believes that voters often follow the peak-end rule according to the theory of behavioral economics when they make voting decisions. However, at present, there is no relevant research on the relationship between economic voting and peak-end rule. This paper discusses voters’ decision process of using peak-end rule with a cross-nation panel data including 595 national elections in 70 countries from 1960 - 2020. The results show that voters do follow the peak-end rule. The average of highest annual GDP growth rate in the term and the GDP growth rate on final year before the election have positive impact on the vote share of the incumbent party. This paper also finds that voters with lower education level rely more on the peak-end rule. To solve the endogenous problems, this paper introduces an instrumental variable strategy. We introduce the global oil price and the federal fund rate as instrumental variables. "
my_title = ""
my_keywords = ""

In [16]:
similar_abstracts_docs = get_similar_docs(my_abstract, 5, embedding_function, collection_name_abstract, persist_directory)

Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index_all


In [17]:
abstracts_output = get_chain_output(my_abstract,similar_abstracts_docs)

In [18]:
filename_list = get_filename_list(abstracts_output, path)

In [ ]:
# save_pdfs_to_db(filename_list, intro_excel_file, is_Intro=True, pages=2)

In [10]:
intros = get_metadata_from_db(intro35_excel_file)

In [ ]:
polish = get_polish_intro('', intros[:3], words_limit=600, temperature=0)

In [12]:
polish

'The three sample introductions provided shed light on different aspects of political behavior and its consequences. The first sample explores the paradox of why people vote despite the low probability of their vote being decisive. The second sample delves into the impact of primary election rules on voting behavior and election outcomes. The third sample investigates the relationship between economic growth and the support for extreme political parties in western democracies. These studies highlight the complexity of political decision-making and the importance of understanding the underlying motivations and incentives that drive it. In this Journal of Economic Literature, we aim to contribute to this understanding by presenting a collection of articles that examine various aspects of political behavior and its economic implications. Through rigorous empirical and theoretical analyses, our contributors shed light on the factors that shape political preferences, the consequences of pol

### 匹配

In [10]:
query = "However, the China's cadre evaluation system is different from the voting system in the Western countries"

In [11]:
similar_docs = get_similar_docs(query, 5, embedding_function, collection_name, persist_directory)

Using embedded DuckDB with persistence: data will be stored in: data/db/vector/huggingface_index2


In [12]:
output = get_chain_output(query,similar_docs)

In [13]:
final_list = run_text_match(output,query,similar_docs, embedding_function)

No result was found, Using embedding searching strategy!!!
The 0 sentence
Sentence: Empirically, the literature linking China’s cadre eval- uation system and growth produces highly ambiguous and problematic results
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6213207978549135
========.
========.
The 1 sentence
Sentence: After the founding of the People’s Republic of China, the CCP introduced formal methods of evaluating cadres, which were formally linked to their promotion
Source:{'author': 'VICTOR SHIH and CHRISTOPHER ADOLPH and MINGXING LIU', 'year': '2012', 'title': 'Getting Ahead in the Communist Party: Explaining the Advancement of Central Committee Members in China', 'journal': 'American Political Science Review'}
Score:0.6769256411963147
========.
========.
The 2 sent